# SPAN Benchmarks H3K4me1

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

from IPython.display import display
import pandas as pd
from tqdm.auto import tqdm
import seaborn as sns

sns.set_style("whitegrid")
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import numpy as np
import scipy as sp
import os

In [ ]:
def bedl(file):
    try:
        tf = pd.read_csv(file, sep='\t', header=None)
        return tf[2] - tf[1]
    except:
        return np.zeros(0)  # Empty file


def lines(file):
    try:
        tf = pd.read_csv(file, sep='\t', header=None)
        return len(tf)
    except:
        return 0  # Empty file

def d(a, b):
    return a / b if b != 0 else 0

# Hg38 load peaks

In [ ]:
GSE26320_PATH_HG38 = os.path.expanduser('~/data/2023_GSE26320')
GSE26320_CELLS = ['GM12878', 'HMEC', 'HSMM', 'K562', 'NHEK', 'NHLF', 'H1', 'Huvec', 'HepG2']
# GSE26320_CELLS = ['GM12878',  'K562', 'H1']
# GSE26320_MODIFICATIONS = ['CTCF', 'H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me2', 'H3K4me3', 'H3K9ac', 'H4K20me1']
GSE26320_MODIFICATIONS = ['H3K4me1']
GSE26320_REPS = ['rep1', 'rep2']

In [ ]:
def load_peaks_fdr(path, suffix, fdrs):
    df_fdr = pd.DataFrame(columns=['file', 'modification', 'cell', 'replicate', 'fdr', 'peaks', 'avlength'],
                          dtype=object)
    for f in tqdm(os.listdir(path)):
        if suffix not in f:
            continue
        fdr = next((fdr for fdr in fdrs if fdr in f), None)
        cell = next((cc for cc in GSE26320_CELLS if cc in f), None)
        mod = next((m for m in GSE26320_MODIFICATIONS if m in f), None)
        rep = 'rep1' if 'rep1' in f else 'rep2'
        if fdr and cell and rep and mod:
            peaks_path = os.path.join(path, f)
            ps, ls = lines(peaks_path), bedl(peaks_path)
            avls = 0 if ps == 0 else sum(ls) / ps
            df_fdr.loc[len(df_fdr)] = (f, mod, cell, rep, fdr, ps, avls)
    return df_fdr

In [ ]:
df_fdr_macs2 = load_peaks_fdr(os.path.join(GSE26320_PATH_HG38, 'macs2'), '.narrowPeak', ['0.05'])
df_fdr_macs2['file'] = [f'{GSE26320_PATH_HG38}/macs2/{f}' for f in df_fdr_macs2['file']]
df_fdr_macs2['tool'] = 'MACS2'
print('MACS2', len(df_fdr_macs2))

df_fdr_macs2broad = load_peaks_fdr(os.path.join(GSE26320_PATH_HG38, 'macs2'), '.broadPeak', ['0.1'])
df_fdr_macs2broad['file'] = [f'{GSE26320_PATH_HG38}/macs2/{f}' for f in df_fdr_macs2broad['file']]
df_fdr_macs2broad['tool'] = 'MACS2 broad'
print('MACS2 broad', len(df_fdr_macs2broad))

df_fdr_sicer = load_peaks_fdr(os.path.join(GSE26320_PATH_HG38, 'sicer'), 'summary-FDR', ['0.01'])
df_fdr_sicer['file'] = [f'{GSE26320_PATH_HG38}/sicer/{f}' for f in df_fdr_sicer['file']]
df_fdr_sicer['tool'] = 'SICER'
print('SICER', len(df_fdr_sicer))

df_fdr_span = load_peaks_fdr(os.path.join(GSE26320_PATH_HG38, 'span'), '.peak', ['0.05'])
df_fdr_span['file'] = [f'{GSE26320_PATH_HG38}/span/{f}' for f in df_fdr_span['file']]
df_fdr_span['tool'] = 'SPAN'
print('SPAN', len(df_fdr_span))

df_fdr_peaks = pd.concat([df_fdr_macs2, df_fdr_macs2broad, df_fdr_sicer, df_fdr_span])
df_fdr_peaks.sample(5)

In [ ]:
# TOOLS_PALETTE = {'MACS2': 'blue', 'MACS2 broad': 'orange', 'SICER': 'green', 'SPAN': 'red', 'Genes': 'brown'}
TOOLS = ['MACS2', 'MACS2 broad', 'SICER', 'SPAN', 'Genes']
palette = plt.cm.get_cmap('tab10')
TOOLS_PALETTE = {t: palette(i) for i, t in enumerate(TOOLS)}

In [ ]:
plt.figure(figsize=(4, 4))
ax = plt.axes()
g_results = sns.barplot(data=df_fdr_peaks, x='tool', y='peaks', ax=ax,
                        capsize=.2, errwidth=2, edgecolor="black",
                        palette=TOOLS_PALETTE)
ax.xaxis.set_tick_params(rotation=90)
ax.title.set_text('H3K4me1 peaks number')
plt.show()

In [ ]:
print('Load lengths')
ts = []
for file, tool in tqdm(zip(df_fdr_peaks['file'], df_fdr_peaks['tool'])):
    lengths = bedl(file)
    t = pd.DataFrame(dict(tool=[tool] * len(lengths), length=lengths))
    ts.append(t.sample(min(len(t), 10_000)))
t = pd.concat(ts).reset_index(drop=True)
del ts
t.sample(10)

In [ ]:
plt.figure(figsize=(4, 4))
ax = plt.axes()
g_results = sns.boxplot(data=t, x='tool', y='length', ax=ax)
ax.title.set_text('H3K4me1 peaks length')
ax.xaxis.set_tick_params(rotation=90)
ax.set_ylim([0, 10_000])
plt.show()

In [ ]:
plt.figure(figsize=(8, 5))
ax = plt.axes()
g_results = sns.histplot(data=t, x='length', hue='tool', ax=ax,
                         stat='density', common_bins=False, common_norm=False,
                         bins=50, kde=True, log_scale=True, alpha=0.2,
                         palette=TOOLS_PALETTE)
g_results.set(xscale='log')
g_results.set_ylim(0, 3)
g_results.set_xlim(1e2, 2e4)
ax.title.set_text('H3K4me1 peaks length')
plt.show()

In [ ]:
# Randomly tweak SPAN,SICER lengths for visualization purposes
tspan = t['tool'] == 'SPAN'
t.loc[tspan, 'length'] += np.random.normal(0, 100, size=sum(tspan))
tsicer = t['tool'] == 'SICER'
t.loc[tsicer, 'length'] += np.random.normal(0, 100, size=sum(tsicer))

In [ ]:
plt.figure(figsize=(8, 5))
ax = plt.axes()
g_results = sns.histplot(data=t, x='length', hue='tool', ax=ax,
                         stat='density', common_bins=False, common_norm=False,
                         kde=True, log_scale=True, alpha=0.2,
                         palette=TOOLS_PALETTE)
g_results.set(xscale='log')
g_results.set_ylim(0, 3)
g_results.set_xlim(1e2, 2e4)
ax.title.set_text('H3K4me1 peaks length')
plt.show()

## Consistency analysis between replicates

In [ ]:
import tempfile

def overlap_by_length(file1, file2, min_overlap):
    tf = tempfile.mktemp()
    !bedtools intersect -a {file1} -b {file2} -wo > {tf}
    try:
        peaks1 = pd.read_csv(file1, sep='\t', header=None, nrows=1)
        ncol1 = len(peaks1.columns)
    except:
        # Empty file
        ncol1 = None
    try:
        peaks2 = pd.read_csv(file2, sep='\t', header=None, nrows=1)
        ncol2 = len(peaks2.columns)
    except:
        # Empty file
        ncol2 = None

    if ncol1 is not None and ncol2 is not None:
        try:
            overlap = pd.read_csv(tf, sep='\t', header=None)
        except:
            return 0, 0, 0
        overlap = overlap[overlap[ncol1 + ncol2] > min_overlap]
        overlap1 = overlap[[0, 1, 2, ncol1 + ncol2]].groupby([0, 1, 2]).aggregate(sum).reset_index()
        overlap2 = overlap[[ncol1, ncol1 + 1, ncol1 + 2, ncol1 + ncol2]]. \
            groupby([ncol1, ncol1 + 1, ncol1 + 2]).aggregate(sum).reset_index()
        return len(overlap1), len(overlap2), overlap2[ncol1 + ncol2].sum()
    else:
        return 0, 0, 0

In [ ]:
from itertools import product

reps_overlap = pd.DataFrame(columns=['modification', 'cell', 'tool', 'rep1', 'rep2',
                                     'peaks1', 'peaks2', 'overlap', 'peaks1_overlap', 'peaks2_overlap'], dtype=object)

tools = list(sorted(set(df_fdr_peaks['tool'])))
for c, m in tqdm(product(GSE26320_CELLS, GSE26320_MODIFICATIONS)):
    print(c, m)
    tm = df_fdr_peaks[(df_fdr_peaks['cell'] == c) & (df_fdr_peaks['modification'] == m)]
    reps = list(sorted(set(tm['replicate'])))
    for tool in tools:
        for i in range(len(reps)):
            for j in range(i + 1, len(reps)):
                rep1, rep2 = reps[i], reps[j]
                t1 = tm[(tm['tool'] == tool) & (tm['replicate'] == rep1)]
                t2 = tm[(tm['tool'] == tool) & (tm['replicate'] == rep2)]
                file1 = t1['file'].values[0]
                file2 = t2['file'].values[0]
                peaks1 = t1['peaks'].values[0]
                peaks2 = t2['peaks'].values[0]
                overlap1, overlap2, _ = overlap_by_length(file1, file2, 100)
                reps_overlap.loc[len(reps_overlap)] = \
                    (m, c, tool, rep1, rep2, peaks1, peaks2, '100bp', overlap1, overlap2)
                for overlap, overlap_param in [
                    ('1bp', ''),
                    ('50%', ' -f 0.5 ')
                ]:
                    overlap1 = !bedtools intersect -a {file1} -b {file2} -wa -u {overlap_param} | wc -l
                    overlap1 = int(overlap1[0])
                    overlap2 = !bedtools intersect -b {file1} -a {file2} -wa -u {overlap_param} | wc -l
                    overlap2 = int(overlap2[0])

                    reps_overlap.loc[len(reps_overlap)] = \
                        (m, c, tool, rep1, rep2, peaks1, peaks2, overlap, overlap1, overlap2)

reps_overlap['peak1_overlap_fraction'] = [d(x, y) for x, y in zip(reps_overlap['peaks1_overlap'], reps_overlap['peaks1'])]
reps_overlap['peak2_overlap_fraction'] = [d(x, y) for x, y in zip(reps_overlap['peaks2_overlap'], reps_overlap['peaks2'])]
reps_overlap

In [ ]:
t = pd.concat([
    reps_overlap[
        ['modification', 'cell', 'tool', 'peak1_overlap_fraction', 'overlap']
    ].copy().rename(dict(peak1_overlap_fraction='value'), axis=1),
    reps_overlap[
        ['modification', 'cell', 'tool', 'peak2_overlap_fraction', 'overlap']
    ].copy().rename(dict(peak2_overlap_fraction='value'), axis=1)
]).reset_index(drop=True)
t

In [ ]:
print('Ignore outliers')
t = t[t['value'] > 0.5]

In [ ]:
plt.figure(figsize=(len(GSE26320_MODIFICATIONS) * 4, 3))
for k, m in enumerate(GSE26320_MODIFICATIONS):
    ax = plt.subplot(1, len(GSE26320_MODIFICATIONS), k + 1)
    sns.boxplot(data=t[t['modification'] == m], x='tool', y='value', hue='overlap', ax=ax)
    ax.xaxis.set_tick_params(rotation=90)
    ax.set_title(m)
    ax.set_xlabel('Tool')
    if k == 0:
        ax.set_ylabel('Overlap')
    else:
        ax.set_ylabel(None)
    if k == len(GSE26320_MODIFICATIONS) - 1:
        # Put a legend to the right of the current axis
        ax.legend(loc='center left', bbox_to_anchor=(1, 0.81))
    else:
        ax.get_legend().remove()
# plt.savefig(f'{GSE26320_PATH_HG38}/analyze/overlap.pdf', bbox_inches='tight', dpi=300)
plt.show()

In [ ]:
plt.figure(figsize=(6, 2))
ax = plt.axes()
sns.boxplot(data=t[t['overlap'] == '1bp'], y='tool', x='value', ax=ax, palette=TOOLS_PALETTE)
# ax.xaxis.set_tick_params(rotation=90)
ax.set_title('Overlap between H3K4me1 replicates')
ax.set_xlabel('Tool')
ax.set_ylabel('Fraction')
plt.show()

## Analyze peaks and diffs

In [ ]:
import tempfile
from itertools import product

tf = tempfile.mktemp()

bench_df = pd.DataFrame(
    columns=['cell', 'replicate', 'name', 'peaks'],
    dtype=object
)

for c, r in tqdm(product(GSE26320_CELLS, GSE26320_REPS)):
    print(c, r)
    for tool in set(df_fdr_peaks['tool']):
        t = df_fdr_peaks[(df_fdr_peaks['tool'] == tool) & (df_fdr_peaks['modification'] == 'H3K4me1') &
                         (df_fdr_peaks['cell'] == c) & (df_fdr_peaks['replicate'] == r)]
        if len(t) == 0:
            continue
        peaks_file = t['file'].values[0]
        peaks = lines(peaks_file)
        bench_df.loc[len(bench_df)] = (c, r, tool, peaks)
# bench_df

In [ ]:
TOOLS = list(sorted(set(bench_df['name'])))

diff_bench_df = pd.DataFrame(
    columns=['cell', 'replicate', 'name', 'peaks'],
    dtype=object
)

tf = tempfile.mktemp()
tf2 = tempfile.mktemp()

for c, r in tqdm(product(GSE26320_CELLS, GSE26320_REPS)):
    print(c, r)
    t = df_fdr_peaks[(df_fdr_peaks['tool'] == 'SPAN') & (df_fdr_peaks['modification'] == 'H3K4me1') &
                     (df_fdr_peaks['cell'] == c) & (df_fdr_peaks['replicate'] == r)]
    if len(t) == 0:
        continue
    span_file = t['file'].values[0]
    # Processing single tools information
    for tool in TOOLS:
        if tool == 'SPAN':
            continue
        t = df_fdr_peaks[(df_fdr_peaks['tool'] == tool) & (df_fdr_peaks['modification'] == 'H3K4me1') &
                         (df_fdr_peaks['cell'] == c) & (df_fdr_peaks['replicate'] == r)]
        if len(t) == 0:
            continue
        peaks_file = t['file'].values[0]
        for name, args in [
            (f'SPAN - {tool}', f' -a {span_file} -b {peaks_file} '),
            (f'{tool} - SPAN', f' -b {span_file} -a {peaks_file} ')]:
            !bedtools intersect {args} -wa -v > {tf}
            peaks = lines(tf)
            diff_bench_df.loc[len(diff_bench_df)] = (c, r, name, peaks)

# display(diff_bench_df.head())

In [ ]:
full_bench_df = pd.concat([
    bench_df[['cell', 'replicate', 'name', 'peaks']],
    diff_bench_df[['cell', 'replicate', 'name', 'peaks']]]).reset_index(drop=True)
full_bench_df.sample(10)

In [ ]:
plt.figure(figsize=(4, 4))
ax = plt.axes()
ax.title.set_text('Peaks number')
g_results = sns.barplot(data=full_bench_df, x='name', y='peaks',
                        ax=ax,
                        capsize=.2, errwidth=2,
                        order=['MACS2', 'MACS2 broad', 'SICER', 'SPAN',
                               'SPAN - MACS2', 'SPAN - MACS2 broad', 'SPAN - SICER',
                               'MACS2 - SPAN', 'MACS2 broad - SPAN', 'SICER - SPAN',
                               ]
                        )
ax.xaxis.set_tick_params(rotation=90)
ax.set_ylabel('Peaks number')
plt.tight_layout()
plt.show()

In [ ]:
# Ignore outliers
display(full_bench_df[full_bench_df['precision'] < 0.1])
t = full_bench_df[full_bench_df['peaks'] > 100].copy()